In [ ]:
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-8f959616-4d92-4073-bbf2-1d3f4f6ffad0',
    'iam_service_endpoint': 'https://iam.bluemix.net/oidc/token',
    'api_key': 'VYkxc-7ohgrp0zdnHRSHAQW-aqI5m-b6j8K3YBllfR4a'
}

configuration_name = 'os_1e498447d5f74cd6b90b92b35bb6514e_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('ATM_CleanData.csv', 'datascienceinbanking-donotdelete-pr-hrnb5icgks2da6'))
data.take(5)


In [ ]:
pandasDF=data.toPandas()
pandasDF.head()

In [ ]:
data = data.withColumnRenamed('Day of Week','DAY_OF_WEEK')
data = data.withColumnRenamed('Time of Day','TIME_OF_DAY')
data = data.withColumnRenamed('Time of Day Band','TIME_OF_DAY_BAND')

In [ ]:
import brunel
%brunel data('pandasDF') treemap x(DAY_OF_WEEK,TIME_OF_DAY_BAND) color(DAY_OF_WEEK) size(FRAUD) sum(FRAUD) tooltip(#all)

In [ ]:
from pixiedust.display import *
display(pandasDF)

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorIndexer, IndexToString
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

SI1 = StringIndexer(inputCol='ATM_POSI', outputCol='positionEncoded')
SI2 = StringIndexer(inputCol='POST_COD_Region',outputCol='regionEncoded')
SI3 = StringIndexer(inputCol='DAY_OF_WEEK',outputCol='dayOfWeekEncoded')
SI4 = StringIndexer(inputCol='TIME_OF_DAY_BAND',outputCol='timeOfDayEncoded')
labelIndexer = StringIndexer(inputCol='FRAUD', outputCol='label').fit(data)

OH1 = OneHotEncoder(inputCol="positionEncoded", outputCol="positionEncoded"+"classVec")
OH2 = OneHotEncoder(inputCol="regionEncoded", outputCol="regionEncoded"+"classVec")
OH3 = OneHotEncoder(inputCol="dayOfWeekEncoded", outputCol="dayOfWeekEncoded"+"classVec")
OH4 = OneHotEncoder(inputCol="timeOfDayEncoded", outputCol="timeOfDayEncoded"+"classVec")

assembler = VectorAssembler(inputCols=["positionEncoded"+"classVec", "regionEncoded"+"classVec", "dayOfWeekEncoded"+"classVec", "timeOfDayEncoded"+"classVec"],\
                            outputCol="features")

In [ ]:

rf=RandomForestClassifier(labelCol="label", featuresCol="features")

labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=labelIndexer.labels)

pipeline = Pipeline(stages=[SI1,SI2,SI3,SI4,labelIndexer,OH1,OH2,OH3,OH4,assembler,rf,labelConverter])

In [ ]:

train, test = data.randomSplit([0.8,0.2], seed=6)
train.cache()
test.cache()

In [ ]:

model = pipeline.fit(train)

In [ ]:
results = model.transform(test)
results=results.select(results["label"],results["prediction"],results["FRAUD"],results['predictedLabel'],results["probability"])
results.toPandas().head(10)

In [ ]:
print('Model precision:' + format(results.filter(results.label == results.prediction).count() / float(results.count())))

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label", metricName="areaUnderROC")
print ('Area under ROC curve = .' + format(evaluator.evaluate(results)))